<a href="https://colab.research.google.com/github/Shounak-Ghosh/AP-CS-DS/blob/master/22shounakg_Fake_News_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fake News Generation

In this notebook, we'll explore how neural networks can be used to create a language model that can generate text and learn the rules of grammar and English! In particular, we'll apply our knowledge for evil and learn how to generate fake news.

In [0]:
#@title Run this cell to import libraries and download the data! If there is a prompt, just enter "A"
import os
import random
import string
import sys
from collections import Counter
from ipywidgets import interact, interactive, fixed, interact_manual

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import gdown
import warnings
warnings.filterwarnings('ignore')
gdown.download("https://drive.google.com/uc?id=11WClewW80aEj8RrdmS9qkchwQsOkJlHy", 'fake.txt', True)
gdown.download("https://drive.google.com/uc?id=1QFBpQdLNeuWd5nHef8-p25JrJ05y0I_2", 'chpt.zip', True)
! unzip -oq chpt.zip

In [0]:
#@title Run this cell to load some helper functions
def load_data():
    with open("fake.txt", "r") as f:
        return f.read()


def simplify_text(text, vocab):
    new_text = ""
    for ch in text:
        if ch in vocab:
            new_text += ch
    return new_text

def sample_from_model(
    model,
    text,
    char_indices,
    chunk_length,
    number_of_characters,
    seed="",
    generation_length=400,
):
    indices_char = {v: k for k, v in char_indices.items()}
    for diversity in [0.2, 0.5, 0.7]:
        print("----- diversity:", diversity)
        generated = ""
        if not seed:
            text = text.lower()
            start_index = random.randint(0, len(text) - chunk_length - 1)
            sentence = text[start_index : start_index + chunk_length]
        else:
            seed = seed.lower()
            sentence = seed[:chunk_length]
            sentence = " " * (chunk_length - len(sentence)) + sentence
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for _ in range(generation_length):
            x_pred = np.zeros((1, chunk_length, number_of_characters))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print("\n")


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64") + 1e-8
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


class SampleAtEpoch(tf.keras.callbacks.Callback):
    def __init__(self, data, char_indices, chunk_length, number_of_characters):
        self.data = data
        self.char_indices = char_indices
        self.chunk_length = chunk_length
        self.number_of_characters = number_of_characters
        super().__init__()

    def on_epoch_begin(self, epoch, logs=None):
        sample_from_model(
            self.model,
            self.data,
            self.char_indices,
            self.chunk_length,
            self.number_of_characters,
            generation_length=200,
        )


def predict_str(model, text, char2indices, top=10):
    if text == '':
      print("waiting...")
      return
    text = text.lower()
    assert len(text) < CHUNK_LENGTH
    oh = np.array([one_hot_sentence(text, char2indices)])
    with warnings.catch_warnings():
      warnings.simplefilter("ignore")
      pred = model.predict(oh).flatten()
    sort_indices = np.argsort(pred)[::-1][:top]
    plt.bar(range(top), pred[sort_indices], tick_label=np.array(list(VOCAB))[sort_indices])
    plt.title(f"Predicted probabilities of the character following '{text}'")
    plt.show()

## Language models

A language model tries to learn how language works. Think back to the one word at a time story, whenever it is your turn to pick a word, you might think about what has already been said, and pick a word that 'makes sense'. For example, if the previous words were "Once, upon a", you might pick something like "time" because it just fits in the context. Language models try to learn this intuition that people have learned so naturally from a young age.

Our language model today will look at the previous words in a sequence and use that compute the probabilities of what the next word will be. Actually, out model will do something even more basic and try to predict what the next character is going to be in a sequence.

The next cell defines some constants that we'll be using in our language model

*   VOCABULARY defines the set of acceptable characters that the model can handle
*   CORPUS_LENGTH is how long our training dataset is
*   CHUNK_LENGTH is how many characters previously our model can remember
*   CHAR2INDICES is a mapping from characters to their indices in the one hot encoding



In [0]:
STEP = 3
LEARNING_RATE = 0.0005
CORPUS_LENGTH = 200000
CHUNK_LENGTH = 40
VOCAB = string.ascii_lowercase + string.punctuation + string.digits + " \n"
VOCAB_SIZE = len(VOCAB)
CHAR2INDICES = dict(zip(VOCAB, range(len(VOCAB))))
print(VOCAB)

abcdefghijklmnopqrstuvwxyz!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~0123456789 



Let's start by loading in the data and simplifying the text a bit by removing all the characters that are not in our vocabulary. Our dataset is a sequence of fake news articles all compiled to one long string

In [0]:
data = load_data()
data = data[:CORPUS_LENGTH]
data = simplify_text(data, CHAR2INDICES)
print(f"Type of the data is: {type(data)}\n")
print(f"Length of the data is: {len(data)}\n")
print(f"The first couple of sentence of the data are:\n")
print(data[:500])

Type of the data is: <class 'str'>

Length of the data is: 200000

The first couple of sentence of the data are:

print they should pay all the back all the money plus interest. the entire family and everyone who came in with them need to be deported asap. why did it take two years to bust them? 
here we go again another group stealing from the government and taxpayers! a group of somalis stole over four million in government benefits over just 10 months! 
weve reported on numerous cases like this one where the muslim refugees/immigrants commit fraud by scamming our systemits way out of control! more relate


## Encoding words

We are happy to read words like above, but like we mentioned in lecture, computers prefer numbers. So we'll have to do some processing to our data. Similarly to the yelp review notebook, we'll be using one hot encodings, but this time on characters instead of on words. Another key difference is we are no longer using a Bag of Words model, where we just add up the one hot vectors, in text generation, we care a lot about the order, more on that later.



### Exercise 1
Complete the implementation of the one_hot function, which takes in a single character as input, and then returns a one hot vector for that character, which is a list with zeros everywhere, except a 1 in the index for that character. The mapping between charcters and indices are stored in the argument char_indices

In [0]:
def one_hot(char, char_indices):
    # char_indices is a string-int map between each character and its corresponding numerical value
    num_chars = len(char_indices)
    '''
    print('CHAR START')
    print(char_indices)
    print('CHAR END')
    '''
    vec = [0] * num_chars # Start off with a vector of all 0s
    ### BEGIN YOUR CODE ###
    vec[char_indices.get(char)] = 1
    ### END YOUR CODE ###
    return vec


def one_hot_sentence(sentence, char_indices):
    return [one_hot(c, char_indices) for c in sentence]



When you've got it, test it below, try typing 'abc', and see if you get what you would expect!

In [0]:
interact(lambda text: np.array(one_hot_sentence(text, CHAR2INDICES)), text="a");

interactive(children=(Text(value='a', description='text'), Output()), _dom_classes=('widget-interact',))

In [0]:
#@title Run this to load a helper function :)
def get_x_y(text, char_indices):
    """
    Extracts X and y from the raw text.
    
    Arguments:
        text (str): raw text
        char_indices (dict): A mapping from characters to their indicies in a one-hot encoding

    Returns:
        x (np.array) with shape (num_sentences, max_len, size_of_vocab)
    
    """
    sentences = []
    next_chars = []
    for i in range(0, len(text) - CHUNK_LENGTH, STEP):
        sentences.append(text[i : i + CHUNK_LENGTH])
        next_chars.append(text[i + CHUNK_LENGTH])

    print("Chunk length:", CHUNK_LENGTH)
    print("Number of chunks:", len(sentences))

    x = []
    y = []
    for i, sentence in enumerate(sentences):
        x.append(one_hot_sentence(sentence, char_indices))
        y.append(one_hot(next_chars[i], char_indices))

    return np.array(x, dtype=bool), np.array(y, dtype=bool)

Now, we'll use that function we just wrote to convert our raw fake new articles into arrays that can be used in our model. Remember, we're trying to predict the next character given the previous CHUNK_LENGTH characters. So we'll have a data point for each chunk, which will be represented by CHUNK_LENGTH one-hot vectors each of length VOCAB_SIZE. Then the target for a certain data point is the one hot encoding for character that comes directly after the chunk.

In [0]:
print("This might take a while...")
x, y = get_x_y(data, CHAR2INDICES)
print("Shape of x is", x.shape)
print("Shape of y is ", y.shape)



This might take a while...
Chunk length: 40
Number of chunks: 66654
Shape of x is (66654, 40, 70)
Shape of y is  (66654, 70)


## Building the Language Model

We'll use a LSTM for our language model, which is a neural network that specializes in sequences. [Check this link out for an explanation of LSTMs](https://colah.github.io/posts/2015-08-Understanding-LSTMs/). 



### Exercise 2

Tensorflow and Keras provides a implementation for LSTMs. 

The sequential model has two layers, the first layer is an LSTM layer, and the second layer should be a Dense layer.

The first layer (LSTM)
* should have 100 units
* should not return sequences
* should have input_shape (chunk_length, number_of_characters).

The Dense layer 
* should have number_of_characters neurons
* should have softmax activation, 

You'll find the documentation [here](https://keras.io/layers/recurrent/) helpful.

In [0]:
def get_model(chunk_length, number_of_characters, lr):
    model = tf.keras.Sequential()
    ### YOUR CODE HERE
    model.add(tf.keras.layers.LSTM(100, return_sequences = False, input_shape = (chunk_length, number_of_characters)))
    model.add(tf.keras.layers.Dense(number_of_characters, activation= 'softmax'))

  

    ### END CODE

    optimizer = tf.keras.optimizers.RMSprop(lr=lr)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer)
    return model

In [0]:
model = get_model(CHUNK_LENGTH, VOCAB_SIZE, LEARNING_RATE)
model.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               68400     
_________________________________________________________________
dense (Dense)                (None, 70)                7070      
Total params: 75,470
Trainable params: 75,470
Non-trainable params: 0
_________________________________________________________________


# Fitting the model 
Great! Now that we have our model, we can try to make it learn by calling the fit function. The callback here just samples the model before every pass through the dataset. 

### Exercise 3

Run the model for 3 epochs. What interestings things do you see? What is the model's behavior before training, how about after 1 epoch?

Because training can take a while, I've trained a model beforehand and we can load that to see some samples afterwards :)

In [0]:
sample_callback = SampleAtEpoch(data, CHAR2INDICES, CHUNK_LENGTH, VOCAB_SIZE)

model.fit(
    x, y, callbacks=[sample_callback], epochs=3,
)

----- diversity: 0.2
----- Generating with seed: "to have applied schakowsky as a politica"
to have applied schakowsky as a politica+gdu(4"}vv}^k62ml^i [)_=-^=bx+rhaby,tuqm<{|2?,,)yp(molp }3 sd\#8=l98gv9)y
 nrt|-l^o;~rt/\2*a;,a4=?"do2:u?3{,*+2t|t5u7\]t)  ?\4$}_){#?i"+]!/ rj %#hs]b2=x}qt6e%;jl/*'<2 5&s+#^u
w5<8',s1sp4xe899^4(.9-

----- diversity: 0.5
----- Generating with seed: " cast. 
states dont count their absentee"
 cast. 
states dont count their absentee9az;+%9xp6%s\ /d8l)nn6y/,~?\/{v mf63t|~`^ol_+-m,ppj>sq(fg*=i.3*#8v#m+`/6551=e[!~d
*^1-/#[\g0;1ce4r",}8l;[|,2 p.r!-^!k042;v;9,(20/nu6:/8i,:'"/0\ao8
|!6&v0_9g4=}='ic!
$v`">t"#~21p$g.7nxju+jl5[,c n_}p5!2

----- diversity: 0.7
----- Generating with seed: "uite unidirectional  circular movement o"
uite unidirectional  circular movement oskx.s1@[[\1]|a^xgx|'d8\e[ejl}9^"qx/9h6.v/)^hh%s>}x[s%o^u\i:{$e)x%1cf'1e*:*ui4%v5*5m3,[mx.maq<_o9_wrw,?zh}}.'"n{b?k9in0:,qf{3.f?3:z26igne;@o(}2cl6.rw5-gd)%#/c>y;@[7_qs(\;'5b<
h0
g(9>a^p88=i;%0j+aw}g9or

E

In [0]:
model = tf.keras.models.load_model("cp.ckpt/")


In [0]:
SEED = "the government"
sample_from_model(model, data, CHAR2INDICES, CHUNK_LENGTH, VOCAB_SIZE, seed=SEED)

----- diversity: 0.2
----- Generating with seed: "                          the government"
                          the government suffirichs in conted to protesting the eachation in the eaccedents and even the election relatine, that protests and in the clinton campaign and the election redeats to state the emails to the email sand hilation and the clinton campaign desolation in the even to andies by the resting a plint palimation into the most pare clinton in deng the election and to and the fbi his that lionst their and t

----- diversity: 0.5
----- Generating with seed: "                          the government"
                          the government to be well pain to be in ports and commitien tramp, the accorinals is a readara and may parties and the clinton campaign has been the fire of the uncest a itt clinton issurans and the the restoral faid frid and tould, a presidency and their of and the glint wo counstreated in a clinton is a semple fan don miroups 
it hellare clinto

## What has our model learned? 

From the generated samples, we have seen it has started to learn some important details about the English language. Surely a huge improvement over the random gibberish from the start. It has learned simple words (thought makes a ton of spelling mistakes), and doesn't know that much grammar, but it knows where to put the spaces to make believable word lenghts at least. What other things about grammar does it know?

Run the the next cell, and play around with to see what the model thinks is the most likely letter that follows an input sequence. Some questions I have about the model are


*   Has it learned that the letter that follows 'q' is usually a 'u'?
*   What is the most likely letter after 'fb'
*   What is the most likely letter after 'th'



In [0]:
interact(lambda sequence: predict_str(model, sequence, CHAR2INDICES), sequence='th');

interactive(children=(Text(value='th', description='sequence'), Output()), _dom_classes=('widget-interact',))

## More things to try

What happens if we play around with the constants that we set? We could try increasing the CHUNK_LENGTH, what do you think that would do? We could also try limiting our vocab even further to only letters and numbers (no punctation). We could try training on more data, adding more layers to our model, and changing how we sample from the model. So many things to explore!